In [1]:
# Install packages
!pip install -qU langchain langchain-google-genai google-generativeai==0.8.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.6 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get("GEMINI_API_KEY")
print(" API key set.")


 API key set.


In [3]:
from typing import Dict
from datetime import datetime
from zoneinfo import ZoneInfo

from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent


In [4]:
# Simple mock dataset (extend or swap with your API later)
MOCK_WEATHER: Dict[str, Dict[str, str]] = {
    "mumbai":  {"condition": "Humid, partly cloudy", "temp_c": "31", "humidity": "75%", "rain_chance": "40%"},
    "delhi":   {"condition": "Hot, dry",             "temp_c": "34", "humidity": "35%", "rain_chance": "10%"},
    "hyderabad":{"condition": "Warm, breezy",        "temp_c": "30", "humidity": "55%", "rain_chance": "20%"},
    "bengaluru":{"condition": "Mild, cloudy",        "temp_c": "26", "humidity": "65%", "rain_chance": "30%"},
}


In [5]:
def _now_ist_str() -> str:
    ist = ZoneInfo("Asia/Kolkata")
    return datetime.now(ist).strftime("%Y-%m-%d %H:%M IST")


In [6]:
@tool
def get_weather(city: str) -> str:
    """Return a short, human-friendly weather report for the given city (mock data)."""
    if not city or not city.strip():
        return "Please provide a city name."
    key = city.strip().lower()
    data = MOCK_WEATHER.get(key)
    if not data:
        # Sensible default when city isn't in our mock set
        data = {"condition": "Partly cloudy", "temp_c": "30", "humidity": "60%", "rain_chance": "20%"}
    return (
        f"City: {city.strip().title()}\n"
        f"Time: {_now_ist_str()}\n"
        f"Condition: {data['condition']}\n"
        f"Temperature: {data['temp_c']} °C\n"
        f"Humidity: {data['humidity']}\n"
        f"Chance of rain: {data['rain_chance']}"
    )


In [7]:
def build_weather_agent():
    llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0)
    tools = [get_weather]

    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "You are a helpful weather assistant.\n"
         "RULES:\n"
         "1) Always call the get_weather tool to answer weather questions.\n"
         "2) If the city is missing or unclear, ask the user to specify a city.\n"
         "3) Return concise, friendly answers.\n"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ])

    agent = create_tool_calling_agent(llm, tools, prompt)
    return AgentExecutor(agent=agent, tools=tools, verbose=False)

weather_agent = build_weather_agent()
print(" Weather agent ready.")


 Weather agent ready.


In [8]:
def ask(q: str):
    res = weather_agent.invoke({"input": q})
    print(res["output"], "\n")

# Known city
ask("What’s the weather in Mumbai?")

# Another known city
ask("Weather update for Bengaluru, please.")

# Unknown or missing city → agent should ask to clarify
ask("What’s the weather like today?")


It's humid and partly cloudy in Mumbai. The temperature is 31 °C with 75% humidity and a 40% chance of rain. 

Bengaluru weather: Mild and cloudy, 26 °C, 65% humidity, with a 30% chance of rain. 

I need to know which city you're asking about. Could you please tell me the city? 

